In [2]:
#define a new DFT which is unitary
#NOTE: in Beals' ['97] he normalizes by \sqrt{d_\lambda/n!}
#but also notes that a basis change is an equivalence relation on rep'ns
#and each equivalence class contains a unitary representatione
#if each rep'n \rho \in \hat{G} is unitary, then the transformation is unitary
#these representations are not unitary
#to make them unitary, use Weyl's unitary trick
#OPTION 1: use the formula P = \int_G \rho(g)\rho(g)^* dg, and take a square root to find Q s.t. P = Q^2
#OPTION 2: define a new invariant inner product compute an orthonormal basis w.r.t. 
#this inner product by computing the Gram matrix A and using A.gram_schmidt()

In [133]:
#find the \sqrt{n} by extending to degree 2 field L = K[x]/(x^2-n)
def square_root_ext(K,n):
    R.<x> = PolynomialRing(K)
    f = x^2-n; assert f in R
    if f.is_irreducible():
        L.<b> = K.extension(f)
        return sqrt(L(n))
    else:
        return sqrt(K(n))

In [153]:
#find the change-of-basis matrix Q making \rho(g) unitary for all g \in G
from sage.matrix.special import diagonal_matrix
from sage.misc.functional import sqrt
def unitary_change_of_basis(SGA,partition):
    rho = SGA.specht_module(partition).representation_matrix
    group_size = SGA.group().cardinality()
    P = (1/group_size)*sum(rho(g)*rho(g).conjugate().transpose() for g in SGA.group())
    d, L = P.eigenmatrix_left()
    K = SGA.base_ring()
    if not all([sqrt(K(a)) in K for a in d.diagonal()]):
        K.<a> = K.extension(list(set([sqrt(a).minpoly() for a in d.diagonal()])))
    return L.inverse() * diagonal_matrix([sqrt(K(a)) for a in d.diagonal()]) * L

In [42]:
#define the Fourier coefficient at the representation specht_module
#which is the Specht module corresponding to partition
def hat(f,partition,SGA,unitary=False):
    specht_module = SGA.specht_module(partition)
    rho = specht_module.representation_matrix
    if unitary:
        Q = unitary_change_of_basis(SGA,partition)
        unitary_factor = specht_module.dimension()/SGA.group().cardinality()
        sqrt_unitary_factor = square_root_ext(Q.base_ring(),unitary_factor)
        return sqrt_unitary_factor*sum(f(g)*Q.inverse()*rho(g)*Q for g in SGA.group())
    else:
        return sum(f(g)*rho(g) for g in SGA.group())

In [5]:
#define the delta function delta_s(t) = {1 if s == t, 0 otherwise}
delta = lambda s: lambda t: 1 if t == s else 0

In [6]:
#for each basis element g \in G compute the Fourier coefficients \hat{\delta_g}(partition) for all partitions
from sage.misc.flatten import flatten
def unitary_dft(SGA):
    unitary_dft_matrix = matrix([flatten([hat(delta(g),partition,SGA,unitary=True).list() for partition in Partitions(SGA.group().degree())]) for g in G]).transpose()
    if unitary_dft_matrix.base_ring() == SR:
        return unitary_dft_matrix.simplify_full()
    else:
        return unitary_dft_matrix

In [55]:
#QUESTION: is the SGA DFT unitary?

In [140]:
SGA = SymmetricGroupAlgebra(QQ,3)

In [137]:
G = SGA.group()

In [138]:
SGA_dft = SGA.dft(); SGA_dft

[   1    1    1    1    1    1]
[   1  1/2   -1 -1/2 -1/2  1/2]
[   0  3/4    0  3/4 -3/4 -3/4]
[   0    1    0   -1    1   -1]
[   1 -1/2    1 -1/2 -1/2 -1/2]
[   1   -1   -1    1    1   -1]

In [100]:
#check if A*A^T == Id. it's not, but the columns are orthonormal
SGA_dft*SGA_dft.transpose()

[  6   0   0   0   0   0]
[  0   3   0   0   0   0]
[  0   0 9/4   0   0   0]
[  0   0   0   4   0   0]
[  0   0   0   0   3   0]
[  0   0   0   0   0   6]

In [101]:
partition = Partitions(SGA.group().degree())[1]; partition

[2, 1]

In [142]:
specht_module = SGA.specht_module(partition)

In [143]:
rho = specht_module.representation_matrix

In [144]:
group_size = SGA.group().cardinality()

In [145]:
P = (1/group_size)*sum(rho(g)*rho(g).conjugate().transpose() for g in SGA.group()); P.base_ring()

Rational Field

In [146]:
d, L = P.eigenmatrix_left()

In [147]:
all([sqrt(a) in QQbar for a in d.diagonal()])

True

In [148]:
K.<a> = NumberField([sqrt(a).minpoly() for a in d.diagonal()]); K

Number Field in a0 with defining polynomial x^2 - 2 over its base field

In [149]:
diagonal_matrix([sqrt(K(a)) for a in d.diagonal()])^2

[  2   0]
[  0 2/3]

In [154]:
Q = unitary_change_of_basis(SGA,partition); Q

[-1/2*a0 - 1/2*a1  1/2*a0 - 1/2*a1]
[ 1/2*a0 - 1/2*a1 -1/2*a0 - 1/2*a1]

In [126]:
unitary_factor = specht_module.dimension()/SGA.group().cardinality(); unitary_factor

1/3

In [127]:
K

Number Field in a0 with defining polynomial x^2 - 2 over its base field

In [68]:
R.<x> = PolynomialRing(K)

In [69]:
f = x^2 - unitary_factor

In [72]:
f.is_irreducible()

False

In [83]:
square_root_ext(K,unitary_factor)

Number Field in a0 with defining polynomial x^2 - 2 over its base field

In [128]:
hat(delta(G[3]),partition,SGA,unitary=True).base_ring()

Algebraic Field

In [156]:
[flatten([hat(delta(g),partition,SGA,unitary=True).list() for partition in Partitions(SGA.group().degree())]) for g in G]

[[b, -1/2*a1*a0, 0, 0, -1/2*a1*a0, b],
 [b, 0, -1/2*a1*a0, -1/2*a1*a0, 0, -b],
 [b, -1/2, 1/4*a1*a0, 1/4*a1*a0, 1/2, -b],
 [b, 1/4*a1*a0, -1/2, 1/2, 1/4*a1*a0, b],
 [b, 1/4*a1*a0, 1/2, -1/2, 1/4*a1*a0, b],
 [b, 1/2, 1/4*a1*a0, 1/4*a1*a0, -1/2, -b]]

In [137]:
G[3]

[2, 3, 1]

In [136]:
rho(G[3])

[6 1]
[6 0]

In [134]:
U_rho = Q.inverse()*rho(G[3])*Q; U_rho

[6 1]
[6 0]

In [139]:
U_rho*conjugate_transpose_pos_char(U_rho)

7


[0 6]
[1 6]

In [131]:
unitary_dft(SymmetricGroupAlgebra(QQbar,3))

[ 0.4082482904638630?  0.4082482904638630?  0.4082482904638630?  0.4082482904638630?  0.4082482904638630?  0.4082482904638630?]
[ 0.5773502691896258?                    0   0.500000000000000? -0.2886751345948129? -0.2886751345948129?  -0.500000000000000?]
[             0.?e-18  0.5773502691896258? -0.2886751345948129?   0.500000000000000?  -0.500000000000000? -0.2886751345948129?]
[             0.?e-18  0.5773502691896258? -0.2886751345948129?  -0.500000000000000?   0.500000000000000? -0.2886751345948129?]
[ 0.5773502691896258?                    0  -0.500000000000000? -0.2886751345948129? -0.2886751345948129?   0.500000000000000?]
[ 0.4082482904638630? -0.4082482904638630? -0.4082482904638630?  0.4082482904638630?  0.4082482904638630? -0.4082482904638630?]

In [132]:
U_dft = unitary_dft(SGA); U_dft*U_dft.conjugate().transpose()

[1.000000000000000?            0.?e-18            0.?e-18            0.?e-18            0.?e-18            0.?e-18]
[           0.?e-18 1.000000000000000?            0.?e-18            0.?e-18            0.?e-18            0.?e-18]
[           0.?e-18            0.?e-18 1.000000000000000?            0.?e-18            0.?e-18            0.?e-18]
[           0.?e-18            0.?e-18            0.?e-18 1.000000000000000?            0.?e-18            0.?e-18]
[           0.?e-18            0.?e-18            0.?e-18            0.?e-18 1.000000000000000?            0.?e-18]
[           0.?e-18            0.?e-18            0.?e-18            0.?e-18            0.?e-18 1.000000000000000?]

In [188]:
#check that the DFT is unitary
(U_dft*U_dft.transpose()) == identity_matrix(SGA.group().cardinality())

True

In [92]:
#what are the eigenvalues?

In [160]:
U_dft = unitary_dft(SGA)
k.<a> = U_dft.charpoly().splitting_field()
eigs = matrix(k,U_dft).eigenvalues(extend=False); eigs

[4*a^5 + 5*a^4 + a^3 + 5*a^2 + 3*a, a^5 + 5*a^4 + 3*a^2 + a, 3*a^5 + 6*a^4 + 4*a^3 + 6*a + 5, 5*a^5 + 6*a^4 + 3*a^3 + 3*a^2 + 3*a + 5, 4*a^5 + 6*a^4 + 2*a^3 + 3*a^2 + 2*a + 4, 3*a^5 + 6*a^4 + 4*a^2 + 4*a + 1]

In [ ]:
eigs

In [93]:
#n=3: two real, two complex
#n=4: all complex
#the magnitude is not 1, they're closely grouped around 2 or 3

In [ ]:
eigs = A.eigenvalues(); eigs

In [ ]:
[abs(eig) for eig in eigs]

In [ ]:
#note that the singluar values are the square roots of the diagonal entries of the Gram matrix
print(SymmetricGroup(n).algebra(CDF).dft().SVD()[1].numpy().diagonal())
print(sqrt((A*A.transpose()).numpy().diagonal()))